In [29]:
import pandas as pd
em=pd.read_csv('Pumadataset_cleaned.csv')
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_df=0.95,min_df=2,stop_words='english')

In [35]:
# Remove excluded words from the 'normalized_text' column
excluded_words = ['bliblimegabrandfestival', 'ebay', 'ad', 'bids', 'united', 'phonewhatsapp', 'dm', 'gt', 'gtgtgt', 'core','pls','th','golf',
                  'poshmark','shopmycloset','closet','added','home','earps','dunk','low','atc','use','select','options','polo','blue',
                  'sickkicks','black','got','mary','want','product','school','goalkeeper','england','ready','pt','theyre',
                  'men', 'woman', 'hombre', 'zapatos', 'listing', 'football', 'kit', 'king', 'like', 'year', 'team', 'make',
                  'new', 'upto', 'amp', 'love', 'world', 'friend', 'day', 'play', 'deal', 'ford', 'sheffield', 'wednesday',
                  'ampxampxfampxampx','ampxampxampxampxampxfampxampxeampx','ampxampxampxfampxdampxf','sunsandsampsportslevelshoesninjapumasssportssssports'
                  'shopee','shopeemy'
]

em['lemmatized_text'] = em['lemmatized_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in excluded_words]))

#Define TF-IDF vectorizer with updated text data
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Fit transform the updated data
dtm = tfidf.fit_transform(em['lemmatized_text'])
dtm

# Initialize NMF model
from sklearn.decomposition import NMF
nmf_model = NMF(n_components=15, random_state=1000)
nmf_model.fit(dtm)

# Function to display topics
def display_topics(model, feature_names, num_top_words):
    for index, topic in enumerate(model.components_):
        print(f"({index}, (", end='')
        top_features = [(feature_names[i], topic[i]) for i in topic.argsort()[-num_top_words:]]
        print(' + '.join([f'{weight:.3f}*" {word} "' for word, weight in reversed(top_features)]), end='')
        print("))")

# Display the topics
num_top_words = 10
feature_names = tfidf.get_feature_names_out()
display_topics(nmf_model, feature_names, num_top_words)


(0, (5.861*" shirt " + 2.208*" puma " + 0.596*" away " + 0.445*" look " + 0.441*" city " + 0.387*" short " + 0.344*" season " + 0.332*" sleeve " + 0.300*" logo " + 0.283*" mil "))
(1, (1.684*" dapper " + 1.683*" varsity " + 1.628*" green " + 1.542*" suede " + 0.871*" available " + 0.771*" sneakerscouts " + 0.346*" puma " + 0.138*" retail " + 0.030*" xl " + 0.030*" athletic "))
(2, (3.111*" shoe " + 0.918*" puma " + 0.345*" running " + 0.196*" sport " + 0.134*" flat " + 0.123*" starting " + 0.110*" pair " + 0.109*" wear " + 0.107*" link " + 0.100*" basketball "))
(3, (3.270*" best " + 0.658*" puma " + 0.194*" boot " + 0.117*" footwear " + 0.081*" car " + 0.078*" song " + 0.076*" lpgatour " + 0.076*" womensgolf " + 0.069*" player " + 0.068*" time "))
(4, (1.461*" buy " + 1.459*" offer " + 1.421*" fashion " + 1.367*" golfapparel " + 1.308*" pgatour " + 0.670*" fusion " + 0.648*" ignite " + 0.347*" puma " + 0.339*" proadapt " + 0.338*" alphacat "))
(5, (1.914*" deliver " + 1.874*" countryw

In [38]:
topic_results=nmf_model.transform(dtm)
topic_results.argmax(axis=1)

array([ 8, 13,  8, ...,  9,  9,  8], dtype=int64)

In [39]:
em['Topic']=topic_results.argmax(axis=1)

In [43]:
mytopic_dict={
    0:'Comfortability',
    1:'Durability',
    2:'Durability',
    3:'Durability',
    4:'Discount and Promotion',
    
    5:'Price',
    6:'Discount and Promotion',
    7:'Size Availability',
    8:'Size Availability',
    9:'Price',

    10:'Price',
    11:'Price',
    12:'Comfortability',
    13:'Comfortability',
    14:'Discount and Promotion'
}

em['Topic Label']=em['Topic'].map(mytopic_dict)

In [44]:
# Extract top ten keywords for each topic
def get_top_keywords(model, feature_names, n_top_words):
    top_keywords = {}
    for idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[-n_top_words:]]
        top_keywords[idx] = ', '.join(reversed(top_features))
    return top_keywords


In [45]:
#DATA PREPROCESSING
# Apply the function to extract top ten keywords
top_keywords = get_top_keywords(nmf_model, tfidf.get_feature_names_out(), 10)

# Add top ten keywords to the DataFrame
em['Top Ten Keywords'] = em['Topic'].apply(lambda x: top_keywords[x])
# Count the frequency of each aspect
aspect_counts = em['Topic Label'].value_counts()

# Display the aspect counts
print("Aspect Counts:")
print(aspect_counts)

Aspect Counts:
Topic Label
Comfortability            3569
Durability                3285
Size Availability         3081
Price                     2603
Discount and Promotion    1293
Name: count, dtype: int64


In [46]:
pd.set_option('display.max_colwidth', None)
# Display the modified DataFrame
df = pd.DataFrame(em[['Topic', 'Topic Label', 'Top Ten Keywords']])
df.tail(5)

,Topic,Topic Label,Top Ten Keywords
13826,9,Price,"check, rm, shopee, running, unisex, white, discount, lazada, sleeve, price"
13827,9,Price,"check, rm, shopee, running, unisex, white, discount, lazada, sleeve, price"
13828,9,Price,"check, rm, shopee, running, unisex, white, discount, lazada, sleeve, price"
13829,9,Price,"check, rm, shopee, running, unisex, white, discount, lazada, sleeve, price"
13830,8,Size Availability,"available, store, puma, online, shop, fenty, instore, collection, creeper, delivery"


In [47]:
df.iloc[120910:120926]

,Topic,Topic Label,Top Ten Keywords


In [48]:
df = pd.DataFrame(em[['created_at','username','lemmatized_text', 'Topic Label']])
df.tail()

,created_at,username,lemmatized_text,Topic Label
13826,Tue Nov 14 03:42:37 +0000 2023,malaysiandaily,check puma unisex redeem profoam running shoe rm get shopee,Price
13827,Tue Nov 14 03:42:26 +0000 2023,malaysiandaily,check puma carina leather shoe white rm get shopee,Price
13828,Tue Nov 14 03:42:15 +0000 2023,malaysiandaily,check puma retaliate tongue running shoe red rm get shopee,Price
13829,Tue Nov 14 03:42:00 +0000 2023,malaysiandaily,check puma retaliate tongue running shoe rm get shopee,Price
13830,Mon Nov 13 22:51:48 +0000 2023,JessiJones3701,ever aware good future weapon whose,Size Availability


In [49]:
# Save the updated DataFrame to a new CSV file
df.to_csv('Pumadataset_aspects.csv', index=False)  # This will save the file in the same directory